In [24]:
# Import necessary libraries
import sqlite3
import pandas as pd


# Connect to a SQLite database (or create one)
conn = sqlite3.connect(r'C:\Users\spiri\codingbootcamp\Final-project-C4Y\data.db')
cursor = conn.cursor()

WITH mean_deposite as (
    select client_id, AVG(balance), currency
    from balances
    group by 1
)
Select * 
from inv_campaign_eval
left join client
on inv_campaign_eval.client_id = client.client_id
left join client_products
on inv_campaign_eval.client_id = client_products.client_id
left join mean_deposite
on inv_campaign_eval.client_id = mean_deposite.client_id

In [25]:
query = ('''
WITH mean_deposite as (
    select client_id, AVG(balance) as mean_balance, currency
    from balances
    group by 1
)
Select * 
from inv_campaign_eval
left join client
on inv_campaign_eval.client_id = client.client_id
left join client_products
on inv_campaign_eval.client_id = client_products.client_id
left join mean_deposite
on inv_campaign_eval.client_id = mean_deposite.client_id''')
df = pd.read_sql_query(query, conn)

df.head()

,client_id,poutcome,client_id,age,job,marital,education,gender,client_id,has_deposits,loan,has_insurance,has_mortgage,client_id,mean_balance,currency
0,249789938,success,249789938,38.0,services,married,secondary,M,249789938,yes,no,n,no,249789938,2669.532734,CZK
1,1504633819,failure,1504633819,38.0,management,married,tertiary,M,1504633819,no,yes,n,yes,1504633819,5440.862334,CZK
2,1952195738,success,1952195738,34.0,services,married,secondary,M,1952195738,yes,yes,n,no,1952195738,2922.958852,CZK
3,989726384,success,989726384,62.0,retired,married,secondary,F,989726384,yes,no,n,no,989726384,3124.614843,CZK
4,1971878545,failure,1971878545,36.0,management,single,tertiary,M,1971878545,yes,no,n,no,1971878545,474.083033,CZK


In [29]:
print(df.columns)
print(df.dtypes)


Index(['client_id', 'poutcome', 'age', 'job', 'marital', 'education', 'gender',
       'has_deposits', 'loan', 'has_insurance', 'has_mortgage', 'mean_balance',
       'currency'],
      dtype='object')
client_id        object
poutcome         object
age              object
job              object
marital          object
education        object
gender           object
has_deposits     object
loan             object
has_insurance    object
has_mortgage     object
mean_balance     object
currency         object
dtype: object


Here we drop duplicate columns

In [27]:
df = df.T.drop_duplicates().T

In [28]:
df.head()

,client_id,poutcome,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,mean_balance,currency
0,249789938,success,38.0,services,married,secondary,M,yes,no,n,no,2669.532734,CZK
1,1504633819,failure,38.0,management,married,tertiary,M,no,yes,n,yes,5440.862334,CZK
2,1952195738,success,34.0,services,married,secondary,M,yes,yes,n,no,2922.958852,CZK
3,989726384,success,62.0,retired,married,secondary,F,yes,no,n,no,3124.614843,CZK
4,1971878545,failure,36.0,management,single,tertiary,M,yes,no,n,no,474.083033,CZK


checking and dealing with the missing value

In [30]:
df.isna().sum()

client_id         0
poutcome          0
age              50
job              83
marital           0
education         0
gender            0
has_deposits      0
loan              0
has_insurance     0
has_mortgage      0
mean_balance      0
currency          0
dtype: int64

In [ ]:
df.job.fillna(value='unknown', inplace=True)

In [ ]:
df.age.fillna(value=df.age.mean(), inplace=True)


So we fill the missing values, no we can focus on the data types

In [53]:
print(df.dtypes)

client_id          int32
poutcome          object
age              float64
job               object
marital           object
education         object
gender            object
has_deposits      object
loan              object
has_insurance     object
has_mortgage      object
mean_balance     float64
currency          object
dtype: object


In [50]:
df.mean_balance = pd.to_numeric(df.mean_balance)

In [52]:
df.client_id = pd.to_numeric(df.client_id)

next part I want to focus on converting currency everyrhing to czk


In [54]:
rate = {'CZK': 1, 'USD': 23, 'EUR': 25}

df['mean_balance'] = df.apply(lambda row: row['mean_balance'] * rate[row['currency']], axis=1)